In [ ]:
# This project uses GluonTS, a Python toolkit for probabilistic time series modeling.
# Copyright 2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# A copy of the License is located at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# or in the "license" file accompanying this file. This file is distributed 
# on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, 
# either express or implied. See the License for the specific language 
# governing permissions and limitations under the License.

# This file uses the GluonTS library for probabilistic time series modeling.
# See https://github.com/awslabs/gluonts for more information.

import mariadb
import pandas as pd
import numpy as np
from gluonts.dataset.common import ListDataset
from datetime import datetime
import pickle

# Set up database connection parameters
config = {
    'host': '#yuor host',
    'user': '#yuor user',
    'password': '#yuor password',
    'port': #yuor port,
    'database': 'deepar_train'
}

# Establish database connection
try:
    conn = mariadb.connect(**config)
    print("Connection successful")
except mariadb.Error as e:
    print(f"Connection failed: {e}")
    exit(1)

# Create a cursor object
cursor = conn.cursor()

# Define the table name to be extracted
tables = ['deepar_validation_dataframe']

dataframes = {}

# Extract data from each table and store it in a dictionary
for table in tables:
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    result = cursor.fetchall()
    column_names = [i[0] for i in cursor.description]
    dataframes[table] = pd.DataFrame(result, columns=column_names)

# Close cursor and connection
cursor.close()
conn.close()

# Process data from each table
for table_name, df in dataframes.items():
    # Convert Date column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Generate item_id
    df['item_id'] = df['Area']

    # Generate start date based on Area
    start_dates = df.groupby('Area')['Date'].min().reset_index()
    df = df.merge(start_dates, on='Area', suffixes=('', '_start'))
    df = df.rename(columns={'Date_start': 'start'})

    # Generate target and observed_values
    df = df.sort_values(by=['item_id', 'Date'])
    df['observed_values'] = 1
    df = df.set_index(['item_id', 'Date'])

    # Fill missing values
    df['VALUE'] = df['VALUE'].fillna(0)
    df['observed_values'] = np.where(df['observed_values'] == 0, 0, 1)
    df['month'] = df['month'].fillna(0)
    df['month_sin'] = df['month_sin'].fillna(0)
    df['month_cos'] = df['month_cos'].fillna(0)
    df['Combined_Recovery_Rate'] = df['Combined_Recovery_Rate'].fillna(0)
    df['cumulative_trend_part'] = df['cumulative_trend_part'].fillna(0)
    df['sales_gradient'] = df['sales_gradient'].fillna(0)
    df['total_fossil'] = df['total_fossil'].fillna(0)
    df['total_sales'] = df['total_sales'].fillna(0)
    df['Combined_Transportation'] = df['Combined_Transportation'].fillna(0)
    df['Iron_Steel_Products'] = df['Iron_Steel_Products'].fillna(0)

    # Handle missing values in dynamic features
    df['month'] = df['month'].fillna(-1)
    df['month_sin'] = df['month_sin'].fillna(-1)
    df['month_cos'] = df['month_cos'].fillna(-1)
    df['Combined_Recovery_Rate'] = df['Combined_Recovery_Rate'].fillna(-1)
    df['cumulative_trend_part'] = df['cumulative_trend_part'].fillna(-1)
    df['sales_gradient'] = df['sales_gradient'].fillna(-1)
    df['total_fossil'] = df['total_fossil'].fillna(-1)
    df['total_sales'] = df['total_sales'].fillna(-1)
    df['Combined_Transportation'] = df['Combined_Transportation'].fillna(-1)
    df['Iron_Steel_Products'] = df['Iron_Steel_Products'].fillna(-1)

    # Define static features
    feat_static_cat = [1, 2]  # 1 represents United States of America, 2 represents mtCO2

    # Find the earliest start date
    earliest_start = df['start'].min()

    # Split the table and save
    list_data = []
    for item_id, group in df.groupby('item_id'):
        target = group['VALUE'].values.tolist()
        observed_values = group['observed_values'].values.tolist()
        start = pd.Timestamp(group['start'].iloc[0])
        fill_length = (start.year - earliest_start.year) * 12 + (start.month - earliest_start.month)
        month = group['month'].values.tolist()
        month_sin = group['month_sin'].values.tolist()
        month_cos = group['month_cos'].values.tolist()
        Combined_Recovery_Rate = group['Combined_Recovery_Rate'].values.tolist()
        cumulative_trend_part = group['cumulative_trend_part'].values.tolist()
        sales_gradient = group['sales_gradient'].values.tolist()
        total_fossil = group['total_fossil'].values.tolist()
        total_sales = group['total_sales'].values.tolist()
        Combined_Transportation = group['Combined_Transportation'].values.tolist()
        Iron_Steel_Products = group['Iron_Steel_Products'].values.tolist()

        # Forward fill with -1
        target = [-1] * fill_length + target
        observed_values = [0] * fill_length + observed_values
        month = [-1] * fill_length + month
        month_sin = [-1] * fill_length + month_sin
        month_cos = [-1] * fill_length + month_cos
        Combined_Recovery_Rate = [-1] * fill_length + Combined_Recovery_Rate
        cumulative_trend_part = [-1] * fill_length + cumulative_trend_part
        sales_gradient = [-1] * fill_length + sales_gradient
        total_fossil = [-1] * fill_length + total_fossil
        total_sales = [-1] * fill_length + total_sales
        Combined_Transportation = [-1] * fill_length + Combined_Transportation
        Iron_Steel_Products = [-1] * fill_length + Iron_Steel_Products
        
        feat_dynamic_real = [
            month,
            month_sin,
            month_cos,
            Combined_Recovery_Rate,
            cumulative_trend_part,
            sales_gradient,
            total_fossil,
            total_sales,
            Combined_Transportation,
            Iron_Steel_Products
        ]

        list_data.append({
            "item_id": item_id,
            "start": earliest_start.strftime('%Y-%m-%d %H:%M:%S'),
            "target": target,
            "feat_static_cat": feat_static_cat,  # Fixed United States of America as 1
            "feat_dynamic_real": feat_dynamic_real  # Merged into a two-dimensional list
        })

    train_data = ListDataset(list_data, freq="M")
    variable_filename = table_name.replace(' ', '_')
    output_path = f'F:/DeepAR_models/Data_clean/gluonts_listDataset_{variable_filename}.pkl'
    with open(output_path, 'wb') as f:
        pickle.dump(train_data, f)
    print(f"Dataset generated and saved as {output_path}")